In [ ]:
from tools import *
from models import *
import plotly.graph_objects as go
import plotly.figure_factory as ff
from Bio.SeqUtils import GC
import pickle

import warnings
warnings.filterwarnings('ignore')
#https://github.com/PuYuQian/PyDanQ/blob/master/DanQ_train.py

In [ ]:
#for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)

In [ ]:
#CONSTANTS AND HYPERPARAMETERS (add to yaml)
# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Hyper parameters
num_epochs = 15 #15
batch_size = 100
learning_rate = 0.003 

In [ ]:
dataloaders, target_labels, train_out = load_datas("../data/tf_peaks_50_partial.h5", batch_size)

In [ ]:
#decode sequences
target_labels = [i.decode("utf-8") for i in target_labels]

In [ ]:
num_classes = len(target_labels) #number of classes

model = DanQ(num_classes).to(device)

criterion = nn.BCEWithLogitsLoss() #- no weights

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
model, train_error, test_error, train_fscore, test_fscore = train_model(dataloaders['train'], 
                                                                        dataloaders['valid'], model, 
                                                                        device, criterion,  optimizer, 
                                                                        num_epochs, 
                                                                        "../weights_multimodel_partial_DanQ", 
                                                                        "", verbose=True) 

In [ ]:
model.load_state_dict(torch.load("../weights_multimodel_partial_DanQ/model_epoch_9_.pth"))
model.eval();

In [ ]:
%matplotlib inline
model.to(device);

labels_E, outputs_E = run_test(model, dataloaders['test'], device)

compute_metrics(labels_E, outputs_E)